# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
R. Launhardt  ->  R. Launhardt  |  ['R. Launhardt']
O. Krause  ->  O. Krause  |  ['O. Krause']
A. Kospal  ->  A. Kospal  |  ['A. Kospal']
Arxiv has 54 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2403.04141


extracting tarball to tmp_2403.04141... done.
Retrieving document from  https://arxiv.org/e-print/2403.04211


extracting tarball to tmp_2403.04211...

 done.
Retrieving document from  https://arxiv.org/e-print/2403.04274


...

✔ → 100778:\section{Resolution study}\label{sec:Resolution_Study}
  ↳ 110457:end


Found 126 bibliographic references in tmp_2403.04439/TIP.bbl.
syntax error in line 137: '=' expected
Retrieving document from  https://arxiv.org/e-print/2403.04476


extracting tarball to tmp_2403.04476...

 done.


/tmp/ipykernel_2161/3009462391.py:49: LatexWarning: 2403.04476 did not run properly
Unable to locate Ghostscript on paths
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.04274-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.04274) | **Relative alignment between gas structures and magnetic field in Orion A  at different scales using different molecular gas tracers**  |
|| W. Jiao, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2024-03-08*|
|*Comments*| *11 pages, 8 figures, accepted for publication in A&A*|
|**Abstract**| Context: Magnetic fields can play crucial roles in high-mass star formation. Nonetheless, the significance of magnetic fields at various scales and their relationship with gas structures is largely overlooked. Aims: Our goal is to examine the relationship between the magnetic field and molecular gas structures within the Orion A giant molecular cloud at different scales and density regimes. Methods: We assess the gas intensity structures and column densities in Orion A by utilizing $^{12}$CO, $^{13}$CO, and C$^{18}$O from Nobeyama observations. Through comparing Nobeyama observations with {\it{Planck}} polarization observations on large scales ($\sim0.6$ pc) and JCMT polarization observations on small scales ($\sim0.04$ pc), we investigate how the role of magnetic fields change with scale and density. Results: We find a similar trend from parallel to perpendicular alignment with increasing column densities in Orion A at both large and small spatial scales. Besides, when changing from low-density to high-density tracers, the relative orientation preference changes from random to perpendicular. The self-similar results at different scales indicate that magnetic fields are dynamically important in both cloud formation and filament formation. However, magnetic fields properties at small scales are relative complicated, and the interplay between magnetic field and star-forming activities needs to be discussed case-by-case. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.04355-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.04355) | **VLBI detection of nearby (< 100 pc) young stars: Pilot observations**  |
|| S. A. Dzib, L. Loinard, <mark>R. Launhardt</mark>, J. Ordóñez-Toro |
|*Appeared on*| *2024-03-08*|
|*Comments*| *Accepted for publication in A&A, 12 pages, 4 figures, 6 tables*|
|**Abstract**| To increase the number of sources with Very Long Baseline Interferometry (VLBI) astrometry available for comparison with the Gaia results, we have observed 31 young stars with recently reported radio emission. These stars are all in the Gaia DR3 catalog and were suggested, on the basis of conventional interferometry observations, to be non-thermal radio emitters and, therefore, good candidates for VLBI detections. The observations were carried out with the Very Long Baseline Array (VLBA) at two epochs separated by a few days and yielded 10 detections (a $\sim$30\% detection rate). Using the astrometric Gaia results, we have extrapolated the target positions to the epochs of our radio observations and compared them with the position of the radio sources. For seven objects, the optical and radio positions are coincident within five times their combined position errors. Three targets, however, have position discrepancies above eight times the position errors, indicating different emitting sources at optical and radio wavelengths. In one case, the VLBA emission is very likely associated with a known companion of the primary target. In the other two cases, we associate the VLBA emission with previously unknown companions, but further observations will be needed to confirm this. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.04439-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.04439) | **Episodic eruptions of young accreting stars: the key role of disc  thermal instability due to Hydrogen ionisation**  |
|| S. Nayakshin, et al. -- incl., <mark>A. Kospal</mark> |
|*Appeared on*| *2024-03-08*|
|*Comments*| *submitted to MNRAS; version taking referee's comments into account*|
|**Abstract**| In the classical grouping of large magnitude episodic variability of young accreting stars, FUORs outshine their stars by a factor of $\sim$ 100, and can last for up to centuries; EXORs are dimmer, and last months to a year. A disc Hydrogen ionisation Thermal Instability (TI) scenario was previously proposed for FUORs but required unrealistically low disc viscosity. In the last decade, many intermediate type objects, e.g., FUOR-like in luminosity and spectra but EXOR-like in duration were found. Here we show that the intermediate type bursters Gaia20eae, PTF14jg, Gaia19bey and Gaia21bty may be naturally explained by the TI scenario with realistic viscosity values. We argue that TI predicts a dearth (desert) of bursts with peak accretion rates between $\dot M \sim 10^{-6} M_\odot$/yr and $\dot M \sim 10^{-5} M_\odot$/yr, and that this desert is seen in the sample of all the bursters with previously determined $\dot M$ burst. Most classic EXORs (FUORs) appear to be on the cold (hot) branch of the S-curve during the peak light of their eruptions; thus TI may play a role in this class differentiation. At the same time, TI is unable to explain how classic FUORs can last for up to centuries, and over-predicts the occurrence rate of short FUORs by at least an order of magnitude. We conclude that TI is a required ingredient of episodic accretion operating at R < 0.1 au, but additional physics must play a role at larger scales. Knowledge of TI inner workings from related disciplines may enable its use as a tool to constrain the nature of this additional physics. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.04211-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.04211) | **Partial tidal disruption events: The elixir of life**  |
|| <mark>M. Sharma</mark>, D. J. Price, A. Heger |
|*Appeared on*| *2024-03-08*|
|*Comments*| *22 pages, 27 figures, submitted to MNRAS, comments welcome*|
|**Abstract**| In our Galactic Center, about 10,000 to 100,000 stars are estimated to have survived tidal disruption events, resulting in partially disrupted remnants. These events occur when a supermassive black hole (SMBH) tidally interacts with a star, but not enough to completely disrupt the star. We use the 1D stellar evolution code Kepler and the 3D smoothed particle hydrodynamics code Phantom to model the tidal disruption of 1, 3, and 10 solar mass stars at zero-age (ZAMS), middle-age (MAMS), and terminal-age main-sequence (TAMS). We map the disruption remnants into Kepler in order to understand their post-distribution evolution. We find distinct characteristics in the remnants, including increased radius, rapid core rotation, and differential rotation in the envelope. The remnants undergo composition mixing that affects their stellar evolution. Whereas the remnants formed by disruption of ZAMS models evolve similarly to unperturbed models of the same mass, for MAMS and TAMS stars, the remnants have higher luminosity and effective temperature. Potential observational signatures include peculiarities in nitrogen and carbon abundances, higher luminosity, rapid rotation, faster evolution, and unique tracks in the Hertzsprung-Russell diagram |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.04141-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.04141) | **VLBI Astrometry of Radio Stars to Link Radio and Optical Celestial  Reference Frames: Observing Strategies**  |
|| J. Zhang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-03-08*|
|*Comments*| *9 pages, 4 figures, accepted for publication in the Monthly Notices of the Royal Astronomy Society (MNRAS)*|
|**Abstract**| The Gaia celestial reference frame (Gaia-CRF) will benefit from a close assessment with independent methods, such as Very Long Baseline Interferometry (VLBI) measurements of radio stars at bright magnitudes. However, obtaining full astrometric parameters for each radio star through VLBI measurements demands a significant amount of observation time. This study proposes an efficient observing strategy that acquires double-epoch VLBI positions to measure the positions and proper motions of radio stars at a reduced cost. The solution for CRF link compatible with individual VLBI position measurements is introduced, and the optimized observing epoch scheduling is discussed. Applying this solution to observational data yields results sensitive to sample increase or decrease, yet they remain consistently in line with the literature at the 1-sigma level. This suggests the potential for improvement with a larger sample size. Simulations for adding observations demonstrate the double-epoch strategy reduces CRF link parameter uncertainties by over 30% compared to the five-parameter strategy. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.04476-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.04476) | **Star-spot activity, orbital obliquity, transmission spectrum, physical  properties, and TTVs of the HATS-2 planetary system**  |
|| F. Biagiotti, et al. |
|*Appeared on*| *2024-03-08*|
|*Comments*| *23 pages, 21 figures, Accepted for publication in Astronomy & Astrophysics*|
|**Abstract**| Our aim in this paper is to refine the orbital and physical parameters of the HATS-2 planetary system and study transit timing variations and atmospheric composition thanks to transit observations that span more than ten years and that were collected using different instruments and pass-band filters. We also investigate the orbital alignment of the system by studying the anomalies in the transit light curves induced by starspots on the photosphere of the parent star. We analysed new transit events from both ground-based telescopes and NASA's TESS mission. Anomalies were detected in most of the light curves and modelled as starspots occulted by the planet during transit events. We fitted the clean and symmetric light curves with the JKTEBOP code and those affected by anomalies with the PRISM+GEMC codes to simultaneously model the photometric parameters of the transits and the position, size, and contrast of each starspot. We found consistency between the values we found for the physical and orbital parameters and those from the discovery paper and ATLAS9 stellar atmospherical models. We identified different sets of consecutive starspot-crossing events that temporally occurred in less than five days. Under the hypothesis that we are dealing with the same starspots, occulted twice by the planet during two consecutive transits, we estimated the rotational period of the parent star and, in turn the projected and the true orbital obliquity of the planet. We find that the system is well aligned. We identified the possible presence of transit timing variations in the system, which can be caused by tidal orbital decay, and we derived a low-resolution transmission spectrum. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to locate Ghostscript on paths</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.04386-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.04386) | **Emission lines due to ionizing radiation from a compact object in the  remnant of Supernova 1987A**  |
|| C. Fransson, et al. -- incl., <mark>O. Krause</mark> |
|*Appeared on*| *2024-03-08*|
|*Comments*| *Authors version of manuscript published in Science on 22 Feb 2024*|
|**Abstract**| The nearby Supernova 1987A was accompanied by a burst of neutrino emission, which indicates that a compact object (a neutron star or black hole) was formed in the explosion. There has been no direct observation of this compact object. In this work, we observe the supernova remnant with JWST spectroscopy finding narrow infrared emission lines of argon and sulphur. The line emission is spatially unresolved and blueshifted in velocity relative to the supernova rest frame. We interpret the lines as gas illuminated by a source of ionizing photons located close to the center of the expanding ejecta. Photoionization models show that the line ratios are consistent with ionization by a cooling neutron star or pulsar wind nebula. The velocity shift could be evidence for a neutron star natal kick. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2403.04274.md
    + _build/html/tmp_2403.04274/./OrionA_HRO.png
    + _build/html/tmp_2403.04274/./OrionA_Xi_all.png
    + _build/html/tmp_2403.04274/./OrionA_AM_all.png
    + _build/html/tmp_2403.04274/./OMC1_HRO.png
    + _build/html/tmp_2403.04274/./OMC1_Xi_all.png
    + _build/html/tmp_2403.04274/./OMC1_AM_all.png
    + _build/html/tmp_2403.04274/./OMC23_HRO.png
    + _build/html/tmp_2403.04274/./OMC23_Xi_all.png
    + _build/html/tmp_2403.04274/./OMC23_AM_all.png
exported in  _build/html/2403.04355.md
    + _build/html/tmp_2403.04355/./HD199143_VLA_VLBA_Gaia.png
    + _build/html/tmp_2403.04355/./V875Per_VLA_VLBA_Gaia.png
    + _build/html/tmp_2403.04355/./AILep_VLA_VLBA_Gaia.png
    + _build/html/tmp_2403.04355/./HD22213_VLA_VLBA_Gaia.png
    + _build/html/tmp_2403.04355/./HD285281_VLA_VLBA_Gaia.png
    + _build/html/tmp_2403.04355/./HD62237_VLA_VLBA_Gaia.png
    + _build/html/tmp_2403.04355/./SAO135659_VLA_VLBA_Gaia.png
    + _build/html/tmp_2403.04355/.

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Relative alignment between gas structures and magnetic field in Orion A at different scales using different molecular gas tracers

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2403.04274-b31b1b.svg)](https://arxiv.org/abs/2403.04274)<mark>Appeared on: 2024-03-08</mark> -  _11 pages, 8 figures, accepted for publication in A&A_

</div>
<div id="authors">

W. Jiao, et al. -- incl., <mark>H. Beuther</mark>

</div>
<div id="abstract">

**Abstract:** Magnetic fields can play crucial roles in high-mass star formation. Nonetheless, the significance of magnetic fields at various scales and their relationship with gas structures is largely overlooked. Our goal is to examine the relationship between the magnetic field and molecular gas structures within the Orion A giant molecular cloud at different scales and density regimes. We assess the gas intensity structures and column densities in Orion A by utilizing $^{12}$ CO, $^{13}$ CO, and C $^{18}$ O from Nobeyama observations. Through comparing Nobeyama observations with $_{Planck_}$ polarization observations on large scales ( $\sim0.6$ pc) and JCMT polarization observations on small scales ( $\sim0.04$ pc), we investigate how the role of magnetic fields change with scale and density. We find a similar trend from parallel to perpendicular alignment with increasing column densities in Orion A at both large and small spatial scales. Besides, when changing from low-density to high-density tracers, the relative orientation preference changes from random to perpendicular. The self-similar results at different scales indicate that magnetic fields are dynamically important in both cloud formation and filament formation. However, magnetic fields properties at small scales are relative complicated, and the interplay between magnetic field and star-forming activities needs to be discussed case-by-case.

</div>

<div id="div_fig1">

<img src="tmp_2403.04274/./OrionA_HRO.png" alt="Fig4.1" width="33%"/><img src="tmp_2403.04274/./OrionA_Xi_all.png" alt="Fig4.2" width="33%"/><img src="tmp_2403.04274/./OrionA_AM_all.png" alt="Fig4.3" width="33%"/>

**Figure 4. -** (**a**): Histogram of relative orientations between intensity structures for different tracers and magnetic field in the whole Orion A region. The red, green, blue solid lines represent the angle distribution in the lowest, medium, highest intensity bins.  (**b**): Histogram Shape Parameter calculated in different column density bins in the whole Orion A region. The red solid line is the optimal linear fitting results between $\xi$ and column density. The gray area represents the 1$\sigma$ confidence interval of the fitted line. (**c**): The same as (**b**), but for the relation between AM and column density. (*fig:OrionA*)

</div>
<div id="div_fig2">

<img src="tmp_2403.04274/./OMC1_HRO.png" alt="Fig5.1" width="33%"/><img src="tmp_2403.04274/./OMC1_Xi_all.png" alt="Fig5.2" width="33%"/><img src="tmp_2403.04274/./OMC1_AM_all.png" alt="Fig5.3" width="33%"/>

**Figure 5. -** Same as Fig. \ref{fig:OrionA}, but for the relative orientation in OMC-1. (*fig:OMC1*)

</div>
<div id="div_fig3">

<img src="tmp_2403.04274/./OMC23_HRO.png" alt="Fig6.1" width="33%"/><img src="tmp_2403.04274/./OMC23_Xi_all.png" alt="Fig6.2" width="33%"/><img src="tmp_2403.04274/./OMC23_AM_all.png" alt="Fig6.3" width="33%"/>

**Figure 6. -** Same as Fig. \ref{fig:OrionA}, but for the relative orientation in OMC-2/3. (*fig:OMC2/3*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2403.04274"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\rahms}[4]{#1^{\rm h}#2^{\rm m}#3\mbox{^{\rm s}\mskip-7.6mu. }#4}$
$\newcommand{\decdms}[4]{#1^{\circ}#2'#3\mbox{"\mskip-7.6mu. }#4}$
$\newcommand{\msec}[2]{#1\mbox{"\mskip-7.6mu. }#2}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.0}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.0}$
$\newcommand{\arraystretch}{1.0}$</div>



<div id="title">

# VLBI detection of nearby ($<100$ pc) young stars    

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2403.04355-b31b1b.svg)](https://arxiv.org/abs/2403.04355)<mark>Appeared on: 2024-03-08</mark> -  _Accepted for publication in A&A, 12 pages, 4 figures, 6 tables_

</div>
<div id="authors">

S. A. Dzib, L. Loinard, <mark>R. Launhardt</mark>, J. Ordóñez-Toro

</div>
<div id="abstract">

**Abstract:** To increase the number of sources with Very Long Baseline  Interferometry (VLBI) astrometry available for comparison with the $_ Gaia_$ results,  we have observed 31 young stars with recently reported radio emission.  These stars are all in the $_ Gaia_$ DR3 catalog and were suggested, on the basis on conventional interferometry observations, to be non-thermal radio emitters and, therefore, good candidates for VLBI detections. The observations were carried out with the Very Long Baseline Array (VLBA) at two epochs separated by a few days and yielded 10 detections (a $\sim30$ \% detection rate).  Using the astrometric $_ Gaia_$ results,  we have extrapolated the target positions to the epochs of our radio observations  and compared them with the position of the radio sources.  For seven objects, the optical and radio positions are coincident within five times their combined position errors. Three targets, however, have position discrepancies  above eight times the position errors, indicating different emitting sources at optical and radio wavelengths. In one case, the VLBA emission is very likely associated with a known companion of the primary target. In the other two cases, we associate the VLBA emission with previously unknown companions, but further observations will be needed to confirm this.

</div>

<div id="div_fig1">

<img src="tmp_2403.04355/./HD199143_VLA_VLBA_Gaia.png" alt="Fig1" width="100%"/>

**Figure 1. -** Positions of radio and optical sources related to HD 199143.
The red star indicates the position of the radio source in the first
detected epoch by the VLBA observations. The Cyan dashed line indicates the optical trajectory of the primary component of the HD 199143 stellar
system from 2016.0
to the epoch of detection of the radio source with the VLBA. The optical position
of the star at the VLBA observed epoch is indicated with the cyan square.
The blue square indicates the position of the secondary, HD 199143 B, whose absolute coordinates are unknown; coordinates relative to the primary from the WDS catalog are used to estimate the absolute position.
_ Gaia_ DR3 and VLBA position errors are smaller than the symbol sizes.
The black line is the trajectory followed by the radio source detected
with the VLA  ([Launhardt, Loinard and Dzib 2022]()) , assuming the derived astrometry at
optical wavelengths, from the VLA observed epoch
to the VLBA observed epoch. The black square indicates the
extrapolated position of the VLA radio source at the epoch of the VLBA
detection. The black cross size indicates the VLA positional error.
 (*fig:VVG*)

</div>
<div id="div_fig2">

<img src="tmp_2403.04355/./V875Per_VLA_VLBA_Gaia.png" alt="Fig6.1" width="11%"/><img src="tmp_2403.04355/./AILep_VLA_VLBA_Gaia.png" alt="Fig6.2" width="11%"/><img src="tmp_2403.04355/./HD22213_VLA_VLBA_Gaia.png" alt="Fig6.3" width="11%"/><img src="tmp_2403.04355/./HD285281_VLA_VLBA_Gaia.png" alt="Fig6.4" width="11%"/><img src="tmp_2403.04355/./HD62237_VLA_VLBA_Gaia.png" alt="Fig6.5" width="11%"/><img src="tmp_2403.04355/./SAO135659_VLA_VLBA_Gaia.png" alt="Fig6.6" width="11%"/><img src="tmp_2403.04355/./HD82159_VLA_VLBA_Gaia.png" alt="Fig6.7" width="11%"/><img src="tmp_2403.04355/./HD82558_VLA_VLBA_Gaia.png" alt="Fig6.8" width="11%"/><img src="tmp_2403.04355/./GJ4199_VLA_VLBA_Gaia.png" alt="Fig6.9" width="11%"/>

**Figure 6. -** Positions of radio and optical sources related to our target sources.
The name of source is indicated at the top of each plot.
Red stars indicate the position of the radio source in the first
detected epoch by the VLBA observations.
The black line is the trajectory followed by the radio source detected
with the VLA  ([Launhardt, Loinard and Dzib 2022]()) , assuming the derived astrometry at
optical wavelengths (see Table \ref{tab:am}) from the VLA observed epoch
to the VLBA observed epoch. The black square indicates the
extrapolated position of the VLA radio source at the epoch of the VLBA
detection. The black cross size indicates the VLA positional error.
Cyan line indicates the trajectory of the optical results from 2016.0
to the epoch of detection of the radio source with the VLBA, the position
of the optical source at this epoch is indicated with the cyan square. _ Gaia_ DR3 and VLBA
position errors are smaller than the symbol sizes. Companion relative positions are shown
as blue squares. (*fig:AmP*)

</div>
<div id="div_fig3">

<img src="tmp_2403.04355/./HD22213_ffig1.png" alt="Fig3.1" width="25%"/><img src="tmp_2403.04355/./HD285281_ffig1.png" alt="Fig3.2" width="25%"/><img src="tmp_2403.04355/./AILEP_ffig1.png" alt="Fig3.3" width="25%"/><img src="tmp_2403.04355/./HD62237_ffig1.png" alt="Fig3.4" width="25%"/>

**Figure 3. -** VLBA images of detected radio sources. The images are centered in the position of the radio source as detected in the first epoch (see Table \ref{tab:am}). The name of the stellar source related to the radio sources is labeled in the top-left corner. Images from
left to right correspond to the epochs 1, 2, and the combination of both epochs. Contour levels are -3, 3, 6, 9, 12, and 15 times the noise level of the image as listed in Table \ref{tab:Imr}. The
yellow open ellipse in the bottom-left corner represents the size of the synthesized beam of the image as listed in Table \ref{tab:Imr}. The predicted optical position in epochs 1, and 2 are shown as red and cyan crosses, respectively. For HD 62237 and SAO 135659, these positions fall outside the region shown. (*fig:NF2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2403.04355"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\avc}[1]{\textcolor{magenta}{\textit{[\small #1]}}}$
$\newcommand{\bref}[1]{{ #1}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\lsun}{{\rm L_{\odot}}}$
$\newcommand{\msun}{{\rm M_{\odot}}}$
$\newcommand{\lta}{\la}$
$\newcommand{\gta}{\ga}$
$\newcommand{\be}{\begin{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand{\le}{{L_{\rm Edd}}}$
$\newcommand{\rp}{{R_{\rm ph}}}$
$\newcommand{\rs}{{R_{\rm s}}}$
$\newcommand{\mo}{{\dot M_{\rm out}}}$
$\newcommand{\me}{{\dot M_{\rm Edd}}}$
$\newcommand{\tc}{{t_{\rm C}}}$
$\newcommand{\rc}{{R_{\rm core}}}$
$\newcommand{\mc}{{M_{\rm core}}}$
$\newcommand{\mbh}{{M_{\rm BH}}}$
$\newcommand{\e}{{\dot m_{\rm E}}}$
$\newcommand{\sgra}{Sgr~A^*}$
$\newcommand\del{#1}{ }$
$\newcommand\SNc{#1}{\textcolor{red}{#1}}$
$\newcommand\mearth{{ {\rm M}_{\oplus}}}$
$\newcommand\mj{{ {\rm M}_{\rm J}}}$
$\newcommand\rj{{ {\rm R}_{\rm J}}}$
$\newcommand\St{ {\rm St}}$
$\newcommand{\avc}[1]{\textcolor{magenta}{\textit{[\small #1]}}}$
$\newcommand\av{#1}{\textcolor{blue}{#1}}$
$\newcommand\SNc{#1}{\textcolor{red}{#1}}$
$\newcommand{\bref}[1]{{ #1}}$
$\newcommand\MSunPerYear{~{\rm M_{\odot}}~yr^{-1} }$
$\newcommand\target{Gaia20eae}$
$\newcommand\konkoly{Konkoly Observatory, Research Centre for Astronomy and Earth Sciences,\\Eötvös Loránd Research Network (ELKH), Konkoly-Thege Miklós út 15--17, 1121 Budapest, Hungary}$
$\newcommand\elte{ELTE Eötvös Loránd University, Institute of Physics, Pázmány Péter sétány 1/A, \ 1117 Budapest, Hungary}$
$\newcommand\heidelberg{Max Planck Institute for Astronomy, Königstuhl 17, 69117 Heidelberg, Germany}$
$\title[Thermal Instability and episodic accretion]{Episodic eruptions of young accreting stars: the key role of disc thermal instability due to Hydrogen ionisation.}$
$\author[Nayakshin et al.]{Sergei Nayakshin^{1 \orcidlink{0000-0002-6166-2206}}\thanks{sergei.nayakshin@le.ac.uk}, Fernando Cruz Sáenz de Miera^{2,3,4 \orcidlink{0000-0002-4283-2185}},$
$Ágnes Kóspál^{4,5,6 \orcidlink{0000-0001-7157-6275}},$
$\newauthor$
$Aleksandra \'{C}alovi\'{c}^{1},$
$Jochen Eislöffel^{7} and Douglas N.C. Lin^{8,9}$
$\ ^{1}School of Physics and Astronomy, University of$
$  Leicester, Leicester, LE1 7RH, UK. \ ^2 Institut de Recherche en Astrophysique et Planétologie, Université de$
$Toulouse, UT3-PS, CNRS, CNES, 9 av. du Colonel Roche, \ 31028 Toulouse Cedex 4, France \ ^3\konkoly\ ^4 CSFK, MTA Centre of Excellence, Konkoly-Thege Miklós út 15-17, 1121 Budapest, Hungary\ ^5 \elte\ ^6 \heidelberg\ ^7 Thüringer Landessternwarte Tautenburg, Sternwarte 5, 07778 Tautenburg, Germany\ ^8 Department of Astronomy \& Astrophysics, University of California, Santa Cruz, CA 95064, USA\ ^9 Institute for Advanced Studies, Tsinghua University, Being 100084, China$
$}$
$\date{Accepted XXX. Received YYY; in original form ZZZ}$
$\pubyear{2023}$
$\begin{document}$
$\label{firstpage}$
$\pagerange{\pageref{firstpage}--\pageref{lastpage}}$
$\maketitle$
$\begin{abstract}$
$In the classical grouping of large magnitude episodic variability of young accreting stars, FUORs outshine their stars by a factor of \sim 100, and can last for up to centuries; EXORs are dimmer, and last months to a year. A disc Hydrogen ionisation Thermal Instability (TI) scenario was previously proposed for FUORs but required \bref{unrealistically low} disc viscosity. In the last decade, many intermediate type objects, e.g., FUOR-like in luminosity and spectra but EXOR-like in duration were found.  Here we show that the intermediate type bursters Gaia20eae, PTF14jg, Gaia19bey and Gaia21bty \bref{may be naturally explained} by the TI scenario with realistic viscosity values.  We argue that TI predicts a dearth (desert) of bursts with \bref{peak} accretion rates between 10^{-6}\MSunPerYear  \lesssim \dot M_{\rm burst} \lesssim 10^{-5}\MSunPerYear, and that this desert is seen in the sample of all the bursters with previously determined \dot M_{\rm burst}. Most classic EXORs (FUORs) appear to be on the cold (hot) branch of the S-curve during the peak light of their eruptions; \bref{thus TI may play a role in this class differentiation}.$
$At the same time, TI is unable to explain how classic FUORs can last for up to centuries, \bref{and over-predicts the occurrence rate of short FUORs by at least an order of magnitude}. We conclude that TI is a required ingredient of episodic accretion operating at R\lesssim 0.1~au, but additional physics must play a role at larger scales. \bref{Knowledge of TI inner workings from related disciplines may enable its use as a tool to constrain the nature of this additional physics.}$
$\end{abstract}$
$\begin{keywords}$
$stars:formation -- stars: protostars -- protoplanetary discs$
$\end{keywords}$
$\n\end{document}\end{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand{\le}{{L_{\rm Edd}}}$
$\newcommand{\rp}{{R_{\rm ph}}}$
$\newcommand{\rs}{{R_{\rm s}}}$
$\newcommand{\mo}{{\dot M_{\rm out}}}$
$\newcommand{\me}{{\dot M_{\rm Edd}}}$
$\newcommand{\tc}{{t_{\rm C}}}$
$\newcommand{\rc}{{R_{\rm core}}}$
$\newcommand{\mc}{{M_{\rm core}}}$
$\newcommand{\mbh}{{M_{\rm BH}}}$
$\newcommand{\e}{{\dot m_{\rm E}}}$
$\newcommand{\sgra}{Sgr~A^*}$
$\newcommand\del{#1}$
$\newcommand\SNc{#1}$
$\newcommand\av{#1}$
$\newcommand\SNc{#1}$</div>



<div id="title">

# Episodic eruptions of young accreting stars: the key role of disc thermal instability due to Hydrogen ionisation.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2403.04439-b31b1b.svg)](https://arxiv.org/abs/2403.04439)<mark>Appeared on: 2024-03-08</mark> -  _submitted to MNRAS; version taking referee's comments into account_

</div>
<div id="authors">

S. Nayakshin, et al. -- incl., <mark>A. Kospal</mark>

</div>
<div id="abstract">

**Abstract:** In the classical grouping of large magnitude episodic variability of young accreting stars, FUORs outshine their stars by a factor of $\sim 100$ , and can last for up to centuries; EXORs are dimmer, and last months to a year. A disc Hydrogen ionisation Thermal Instability (TI) scenario was previously proposed for FUORs but required $\bref{unrealistically low}$ disc viscosity. In the last decade, many intermediate type objects, e.g., FUOR-like in luminosity and spectra but EXOR-like in duration were found.  Here we show that the intermediate type bursters Gaia20eae, PTF14jg, Gaia19bey and Gaia21bty $\bref{may be naturally explained}$ by the TI scenario with realistic viscosity values.  We argue that TI predicts a dearth (desert) of bursts with $\bref{peak}$ accretion rates between $10^{-6}$ $\MSunPerYear$ $\lesssim \dot M_{\rm burst} \lesssim 10^{-5}$ $\MSunPerYear$ , and that this desert is seen in the sample of all the bursters with previously determined $\dot M_{\rm burst}$ . Most classic EXORs (FUORs) appear to be on the cold (hot) branch of the S-curve during the peak light of their eruptions; $\bref{thus TI may play a role in this class differentiation}$ .At the same time, TI is unable to explain how classic FUORs can last for up to centuries, $\bref{and over-predicts the occurrence rate of short FUORs by at least an order of magnitude}$ . We conclude that TI is a required ingredient of episodic accretion operating at $R\lesssim 0.1$ au, but additional physics must play a role at larger scales. $\bref{Knowledge of TI inner workings from related disciplines may enable its use as a tool to constrain the nature of this additional physics.}$

</div>

<div id="div_fig1">

<img src="tmp_2403.04439/./Figures/Classic_TI_alphac_0.01.png" alt="Fig1.1" width="33%"/><img src="tmp_2403.04439/./Figures/Classic_TI_alphac_0.03.png" alt="Fig1.2" width="33%"/><img src="tmp_2403.04439/./Figures/Classic_TI_alphac_0.01_Mstar.png" alt="Fig1.3" width="33%"/>

**Figure 1. -** Top: Stellar accretion rate history for five different feeding rates and $\alpha_{\rm c} =0.01$, $M_* = 1.15  $\msun$$. Middle: Same but for $\alpha_{\rm c} = 0.03$. Bottom: Similar to the above but for a fixed $\dot M_{\rm feed} = 1.6\times 10^{-6}$\MSunPerYear, $\alpha_{\rm c} = 0.03$, and several values of $M_*$. Please note that the time axis is different for the three panels. For further detail and discussion see \S\ref{sec:param_space}. (*fig:Lots_of_bursts*)

</div>
<div id="div_fig2">

<img src="tmp_2403.04439/./Figures/Sigma_Temp_TI_burst.png" alt="Fig11" width="100%"/>

**Figure 11. -** Snapshots of the inner disc just before and during one burst of the classic TI corresponding to the blue curve in the middle panel of Fig. \ref{fig:Lots_of_bursts}. The purple dashed lines in the top left panel show the critical disc surface densities (cf. the left panel of Fig. \ref{fig:S-Curve} and eq. 11) on the S-curve as a function of radius. The upper curve corresponds to $\Sigma_{\rm max}$ and the lower one to $\Sigma_{\rm min}$. Above $\Sigma_{\rm max}$ the TI is triggered and the disc travels to the hot (ionised H) stable branch of the S-curve, whereas below $\Sigma_{\rm min}$ the disc falls back to the cold neutral H branch. Note that the instability trigger occurs close to the star and is very rapid. The maximum extent of the TI-affected region is $R_{\rm TI} \sim 0.15$ au. The legend shows the respective times for each snapshot. (*fig:Classic_TI_Sigma_T*)

</div>
<div id="div_fig3">

<img src="tmp_2403.04439/./Figures/Scurve_BLin94_scaled_0_.png" alt="Fig8.1" width="50%"/><img src="tmp_2403.04439/./Figures/Scurve_BLin94_scaled_0_TCO_.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** Left: Equilibrium $\dot M$ vs disc column depth, $\Sigma$, for the Bell \& Lin (1994) opacities,  mass $M_* =0.6$\msun$$, fixed $\alpha=0.01$ and various values of radius $R$. Two critical points where the instability sets in are marked with arrows for $R=0.04$ au curve. This disc annulus is unstable to TI if the long-term feeding rate $\dot M_{\rm feed}$ is between $\dot M_{\rm max}$ and $\dot M_{\rm min}$. Right: the same S-curves but with the central (midplane) disc temperature on the vertical axis. See text in \S\ref{sec:scaling} for details. (*fig:S-Curve*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2403.04439"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

382  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
